<a href="https://colab.research.google.com/github/cherriechang/procedural-learning/blob/main/PL_Graded_ASRT_Transition_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transition Matrix for Graded ASRT Task Design

In [19]:
import numpy as np
import pandas as pd
import itertools

## Graded ASRT Design [4 locations]


### Core design idea

```
Four locations (1, 2, 3, 4), each with different conditional entropy:

Location 1: 0-CHOICE (DETERMINISTIC) (0 bits entropy)
  E.g. after 1: always goes to position 2 (100%)  

Location 2: 2-CHOICE (1 bit entropy)
  E.g. after 2: 2 → 3 (50%) or 2 → 4 (50%)

Location 3: 3-CHOICE (1.58 bits entropy)
  E.g. after 3: 3 → 1 (33%), 3 → 2 (33%), 3 → 4 (33%)

Location 4: 4-CHOICE (MAXIMALLY NONDETERMINISTIC) (2 bits entropy)
  E.g.: after 4: 4 → 1 (25%), 4 → 2 (25%), 4 → 3 (25%), 4 → 4 (25%)
```

### What we get to test

```
1. GRADED LEARNING
   • Can learners extract structure at different entropy levels?
   • Which conditions show fastest learning?
   • At what point does learning start to fail?
   -> What is the dynamic range of the procedural learning system?
   -> Is there a point where uncertainty becomes too high for the procedural system, and learning shifts to a different mechanism?

2. INDIVIDUAL DIFFERENCES
   • Do some people learn deterministic better?
   • Do others handle uncertainty better?
   -> What factors contribute to individual variability in procedural learning?

3. CRITICAL PERIOD EFFECTS
   • Is the developmental trajectory of procedural learning uniform across entropy evels?

4. GRAMMAR CORRELATION SPECIFICITY
   • Does grammar correlate equally with ALL entropy levels?
  ```

### Candidate constraints
(in decreasing order of importance)



```
π is the stationary distribution
P is the transition matrix we are building

1. UNIFORM UNIGRAM DISTRIBUTION [STRICT]
π₁ = π₂ = π₃ = π₄ = 1/4
πP = π
i.e. a doubly stochastic matrix (rows AND columns sum to 1)

2. INCREASING BRANCHING/GRADED ENTROPY ACROSS ROWS [CORE IDEA]
Strong formulation:
- Strictly increasing
- With first row being deterministic (only map to one state)
- And last row being fully undeterministic (can map to any state)

3. UNIFORM (SUPPORTED) TRANSITION PROBABILITIES ON EACH ROW
All values in a row are equal to one another (unless zero)

4. NO SELF LOOPS (REPETITIONS)
Diagonal of transition matrix should be 0-vector
This can be mitigated by throwing out repetition (and/or trills like 1-2-1, 3-1-3) data in post-processing,
which is the approach in some past ASRT papers (e.g. Janacsek 2012)
```

### Functions

In [13]:
def get_transition_matrix(P, state_labels=None):
    n = P.shape[0]
    assert P.shape[0] == P.shape[1], "P must be square"

    if state_labels is None:
        state_labels = list(range(1, n + 1))

    df = pd.DataFrame(
        P,
        index=[f"FROM {s}" for s in state_labels],
        columns=[f"TO {s}" for s in state_labels],
    )
    return df

In [9]:
def get_stability_distribution(P):
  # eigen-decomposition of P^T
  eigvals, eigvecs = np.linalg.eig(P.T)

  # find eigenvector with eigenvalue ~ 1
  idx = np.argmin(np.abs(eigvals - 1))

  pi = np.real(eigvecs[:, idx])
  # normalize
  pi = pi / pi.sum()

  return pi

### V1:
```
P:
            TO 1      TO 2  TO 3      TO 4
FROM 1  0.000000  1.000000  0.00  0.000000
FROM 2  0.000000  0.000000  0.50  0.500000
FROM 3  0.333333  0.333333  0.00  0.333333
FROM 4  0.250000  0.250000  0.25  0.250000

π:
[0.15384615 0.30769231 0.23076923 0.30769231]
```

- Satisfies constraints: 2, 3
- Violates constraint 1: Uneven unigram distribution over time
- Violates constraint 4: Self-loops for state 4



In [15]:
initial_p_v1 = np.array([[0, 1, 0, 0],
          [0, 0, 1/2, 1/2],
          [1/3, 1/3, 0, 1/3],
          [1/4, 1/4, 1/4, 1/4]])
initial_p_df_v1 = get_transition_matrix(initial_p_v1)
initial_p_pi_v1 = get_stability_distribution(initial_p_v1)

print("Initially proposed transition matrix P:")
print(initial_p_df_v1)

print("\nStability Distribution:")
print(initial_p_pi_v1)

Initially proposed transition matrix P:
            TO 1      TO 2  TO 3      TO 4
FROM 1  0.000000  1.000000  0.00  0.000000
FROM 2  0.000000  0.000000  0.50  0.500000
FROM 3  0.333333  0.333333  0.00  0.333333
FROM 4  0.250000  0.250000  0.25  0.250000

Stability Distribution:
[0.15384615 0.30769231 0.23076923 0.30769231]


### V2
```
P:
            TO 1      TO 2      TO 3      TO 4
FROM 1  0.000000  1.000000  0.000000  0.000000
FROM 2  0.000000  0.000000  0.500000  0.500000
FROM 3  0.333333  0.333333  0.000000  0.333333
FROM 4  0.333333  0.333333  0.333333  0.000000

π:
[0.16666667 0.33333333 0.25 0.25]
```
- Satisfies constraints: 3, 4
- Violates constraint 1: Uneven unigram distribution over time
- Violates constraint 2: No entropy increase from state 3 to 4



In [17]:
initial_p_v2 = np.array([[0, 1, 0, 0],
          [0, 0, 1/2, 1/2],
          [1/3, 1/3, 0, 1/3],
          [1/3, 1/3, 1/3, 0]])
initial_p_df_v2 = get_transition_matrix(initial_p_v2)
initial_p_pi_v2 = get_stability_distribution(initial_p_v2)

print("Initially proposed transition matrix P:")
print(initial_p_df_v2)

print("\nStability Distribution:")
print(initial_p_pi_v2)

Initially proposed transition matrix P:
            TO 1      TO 2      TO 3      TO 4
FROM 1  0.000000  1.000000  0.000000  0.000000
FROM 2  0.000000  0.000000  0.500000  0.500000
FROM 3  0.333333  0.333333  0.000000  0.333333
FROM 4  0.333333  0.333333  0.333333  0.000000

Stability Distribution:
[0.16666667 0.33333333 0.25       0.25      ]


### Main conflicts:
- 1 vs strong 2: Cannot have

In [18]:
def entropy(probs):
    """Calculate Shannon entropy"""
    p = probs[probs > 0]
    return -np.sum(p * np.log2(p))

def check_doubly_stochastic(P):
    """Check if matrix is doubly stochastic"""
    return (np.allclose(P.sum(axis=0), 1.0) and
            np.allclose(P.sum(axis=1), 1.0))

def generate_row_variants(n_positions, support_size, exclude_diagonal=True):
    """
    Generate all possible rows with:
    - n_positions total positions
    - support_size non-zero positions
    - equal probabilities on support
    - optionally excluding diagonal
    """
    positions = list(range(n_positions))
    variants = []

    for support in itertools.combinations(positions, support_size):
        # Check if diagonal is in support
        row_idx = len(variants) // len(list(itertools.combinations(positions, support_size)))
        if exclude_diagonal and row_idx in support:
            continue

        row = np.zeros(n_positions)
        prob = 1.0 / support_size
        for pos in support:
            row[pos] = prob
        variants.append(row)

    return variants

n_pos = 4
feasible_matrices = []

# Enumerate all possible row patterns
# Pattern = (support_size_1, support_size_2, support_size_3, support_size_4)
# where support_size_i = number of non-zero entries in row i

# For increasing entropy with constraint #3:
# Must have: support_size_1 ≤ support_size_2 ≤ support_size_3 ≤ support_size_4

for s1 in range(1, n_pos):  # No self-loop means max is n_pos-1 for deterministic
    for s2 in range(s1, n_pos):
        for s3 in range(s2, n_pos):
            for s4 in range(s3, n_pos):
                # Try to construct a doubly stochastic matrix
                # with this support pattern

                # For now, just check if pattern could work
                pattern = (s1, s2, s3, s4)

                # Each column needs total inflow = 1.0
                # Row i contributes 1/s_i to each of its s_i target columns

                # Simplified check: Can we balance columns?
                # This is a constraint satisfaction problem

                # For demonstration, just track promising patterns
                entropies = [np.log2(s) for s in pattern]

                if all(entropies[i] < entropies[i+1] for i in range(3)):
                    # Strictly increasing
                    print(f"\nPattern {pattern}: Entropies = {[f'{e:.2f}' for e in entropies]}")


EXHAUSTIVE SEARCH FOR FEASIBLE DESIGNS

### SEARCH PARAMETERS ###

N = 4 positions
Constraint #1: Doubly stochastic (strict)
Constraint #2: Increasing entropy (strict ordering)
Constraint #3: Uniform supported probabilities (strict)
Constraint #4: No self-loops (strict)

### SEARCHING... ###


======================================================================
EXHAUSTIVE SEARCH FOR FEASIBLE DESIGNS
======================================================================

### SEARCH PARAMETERS ###

N = 4 positions
Constraint #1: Doubly stochastic (strict)
Constraint #2: Increasing entropy (strict ordering)
Constraint #3: Uniform supported probabilities (strict)
Constraint #4: No self-loops (strict)

### SEARCHING... ###

In [ ]:
def search_all_configurations():
    """
    Exhaustively search all possible doubly stochastic matrices
    with uniform supported probabilities and no self-loops
    """
    n = 4
    feasible = []

    print("Searching all valid configurations...")
    print("(This checks all possible ways to assign support sets to rows)")

    # For each row, enumerate possible support sets (excluding diagonal)
    all_row_possibilities = []
    for row_idx in range(n):
        # Possible positions (excluding self-loop)
        other_positions = [j for j in range(n) if j != row_idx]

        # All possible non-empty subsets
        row_options = []
        for size in range(1, len(other_positions) + 1):
            for support in combinations(other_positions, size):
                row = np.zeros(n)
                prob = 1.0 / size
                for pos in support:
                    row[pos] = prob
                row_options.append(row)

        all_row_possibilities.append(row_options)

    # Try all combinations
    total_configs = np.prod([len(opts) for opts in all_row_possibilities])
    print(f"Total configurations to check: {total_configs}")

    count = 0
    for rows in product(*all_row_possibilities):
        P = np.array(rows)

        # Check if doubly stochastic
        if is_doubly_stochastic(P):
            # Check if entropies are increasing
            entropies = [entropy(row) for row in P]

            # Allow non-strictly increasing (≤) to be more permissive
            if all(entropies[i] <= entropies[i+1] for i in range(3)):
                feasible.append({
                    'matrix': P.copy(),
                    'entropies': entropies
                })
                count += 1

    print(f"\nFound {count} feasible configurations!")
    return feasible

# Run search
feasible_designs = search_all_configurations()

print("\n" + "=" * 70)
print(f"FOUND {len(feasible_designs)} FEASIBLE DESIGNS")
print("=" * 70)

if len(feasible_designs) > 0:
    print("\nShowing top designs with distinct entropy profiles...")

    # Group by entropy profile
    seen_profiles = set()
    unique_designs = []

    for design in feasible_designs:
        H = design['entropies']
        profile = tuple(round(h, 3) for h in H)

        if profile not in seen_profiles:
            seen_profiles.add(profile)
            unique_designs.append(design)

    print(f"\nFound {len(unique_designs)} unique entropy profiles\n")

    # Show top 10
    for i, design in enumerate(unique_designs[:10], 1):
        P = design['matrix']
        H = design['entropies']

        print(f"\n{'='*60}")
        print(f"Design {i}:")
        print(f"Entropies: {[f'{h:.3f}' for h in H]} bits")
        print(f"\nTransition matrix P:")
        for j, row in enumerate(P, 1):
            print(f"  Row {j}: {row}")

        # Check properties
        print(f"\n  ✓ Doubly stochastic")
        print(f"  ✓ No self-loops")
        print(f"  ✓ Uniform supported probabilities")
        increasing = "Increasing" if all(H[i] < H[i+1] for i in range(3)) else "Non-decreasing"
        print(f"  {increasing} entropies")

else:
    print("\n⚠️  NO FEASIBLE DESIGNS FOUND")
    print("\nThis means the constraints are too strict!")

EOF
Output

Searching all valid configurations...
(This checks all possible ways to assign support sets to rows)
Total configurations to check: 2401

Found 24 feasible configurations!

======================================================================
FOUND 24 FEASIBLE DESIGNS
======================================================================

Showing top designs with distinct entropy profiles...

Found 5 unique entropy profiles


============================================================
Design 1:
Entropies: ['-0.000', '-0.000', '-0.000', '-0.000'] bits

Transition matrix P:
  Row 1: [0. 1. 0. 0.]
  Row 2: [1. 0. 0. 0.]
  Row 3: [0. 0. 0. 1.]
  Row 4: [0. 0. 1. 0.]

  ✓ Doubly stochastic
  ✓ No self-loops
  ✓ Uniform supported probabilities
  Non-decreasing entropies

============================================================
Design 2:
Entropies: ['-0.000', '1.000', '1.000', '1.000'] bits

Transition matrix P:
  Row 1: [0. 1. 0. 0.]
  Row 2: [0.  0.  0.5 0.5]
  Row 3: [0.5 0.  0.  0.5]
  Row 4: [0.5 0.  0.5 0. ]

  ✓ Doubly stochastic
  ✓ No self-loops
  ✓ Uniform supported probabilities
  Non-decreasing entropies

============================================================
Design 3:
Entropies: ['-0.000', '-0.000', '1.000', '1.000'] bits

Transition matrix P:
  Row 1: [0. 0. 1. 0.]
  Row 2: [0. 0. 0. 1.]
  Row 3: [0.5 0.5 0.  0. ]
  Row 4: [0.5 0.5 0.  0. ]

  ✓ Doubly stochastic
  ✓ No self-loops
  ✓ Uniform supported probabilities
  Non-decreasing entropies

============================================================
Design 4:
Entropies: ['1.000', '1.000', '1.000', '1.000'] bits

Transition matrix P:
  Row 1: [0.  0.5 0.5 0. ]
  Row 2: [0.5 0.  0.  0.5]
  Row 3: [0.5 0.  0.  0.5]
  Row 4: [0.  0.5 0.5 0. ]

  ✓ Doubly stochastic
  ✓ No self-loops
  ✓ Uniform supported probabilities
  Non-decreasing entropies

============================================================
Design 5:
Entropies: ['1.585', '1.585', '1.585', '1.585'] bits

Transition matrix P:
  Row 1: [0.         0.33333333 0.33333333 0.33333333]
  Row 2: [0.33333333 0.         0.33333333 0.33333333]
  Row 3: [0.33333333 0.33333333 0.         0.33333333]
  Row 4: [0.33333333 0.33333333 0.33333333 0.        ]

  ✓ Doubly stochastic
  ✓ No self-loops
  ✓ Uniform supported probabilities
  Non-decreasing entropies
Fascinating! The search found designs, but notice: NONE have strictly increasing entropies across all 4 rows!

Let me analyze this more carefully:

### INSIGHT ###

Even with increasing support sizes, we need to CHECK
if we can actually assign positions to each row
such that columns sum to 1.0

This is non-trivial! Let me search more carefully...